In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/tmp/ipykernel_1820871/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#model_path = "gpt2-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"
model_path = "gpt3-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_100M_whitespace"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    #print(token_surprisals)

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 10.494812965393066),
 ('that', 4.336230278015137),
 ('your', 8.928290367126465),
 ('friend', 6.415864944458008),
 ('gave', 12.488154411315918),
 ('a', 6.056341648101807),
 ('baguette', 39.892873764038086),
 ('to', 6.948508262634277),
 ('Mary', 12.979372024536133),
 ('last', 10.764171600341797),
 ('weekend', 8.105887413024902),
 ('.', 12.205984115600586)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(sentence_units)

In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df

def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    '''    
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')
    '''

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [8]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [9]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['gap_distance'] = pd.Categorical(sentence_df['gap_distance'], categories = ['null', 'short', 'medium', 'long'], ordered = True)

Gap Position == Object

In [10]:
# modifier: short_mod, med_mod, long_mod

In [11]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [12]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 34.5630
Min. group size: 16
Log-Likelihood: -1029.3697
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,34.144,1.319,25.878,0.000,31.558,36.730
1,gap_distance[T.short],0.363,0.930,0.390,0.696,-1.459,2.185
2,gap_distance[T.medium],-0.456,0.930,-0.490,0.624,-2.278,1.366
3,gap_distance[T.long],-1.259,0.930,-1.354,0.176,-3.080,0.563
4,wh_numeric,0.073,0.657,0.111,0.912,-1.215,1.361
5,wh_numeric:gap_distance[T.short],-0.074,0.930,-0.080,0.937,-1.896,1.748
6,wh_numeric:gap_distance[T.medium],-0.029,0.930,-0.031,0.975,-1.851,1.793
7,wh_numeric:gap_distance[T.long],-0.020,0.930,-0.022,0.983,-1.842,1.802
8,gap_numeric,-1.558,0.657,-2.371,0.018,-2.847,-0.270
9,gap_numeric:gap_distance[T.short],0.127,0.930,0.137,0.891,-1.695,1.949



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,26.177,1.615


In [13]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 33.7337
Min. group size: 16
Log-Likelihood: -1052.7529
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,34.460,1.257,27.419,0.000,31.996,36.923
1,wh_numeric,0.047,0.516,0.091,0.928,-0.965,1.059
2,gap_numeric,-1.779,0.516,-3.445,0.001,-2.791,-0.767
3,wh_numeric:gap_numeric,-0.186,0.516,-0.361,0.718,-1.198,0.826
4,gap_distance_length,-0.018,0.011,-1.614,0.106,-0.041,0.004
5,wh_numeric:gap_distance_length,-0.000,0.011,-0.012,0.991,-0.022,0.022
6,gap_numeric:gap_distance_length,0.005,0.011,0.403,0.687,-0.018,0.027
7,wh_numeric:gap_numeric:gap_distance_length,0.002,0.011,0.171,0.864,-0.020,0.024



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,26.207,1.632


Gap Postion == PP/Goal

In [14]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [15]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 71.4454
Min. group size: 16
Log-Likelihood: -1138.2142
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,29.167,1.778,16.400,0.000,25.681,32.653
1,gap_distance[T.short],0.294,1.336,0.220,0.826,-2.325,2.914
2,gap_distance[T.medium],-0.230,1.336,-0.172,0.864,-2.849,2.390
3,gap_distance[T.long],-0.697,1.336,-0.521,0.602,-3.316,1.923
4,wh_numeric,0.159,0.945,0.168,0.866,-1.693,2.011
5,wh_numeric:gap_distance[T.short],-0.071,1.336,-0.053,0.957,-2.691,2.548
6,wh_numeric:gap_distance[T.medium],-0.083,1.336,-0.062,0.951,-2.702,2.537
7,wh_numeric:gap_distance[T.long],-0.074,1.336,-0.056,0.956,-2.694,2.545
8,gap_numeric,3.059,0.945,3.237,0.001,1.207,4.911
9,gap_numeric:gap_distance[T.short],0.176,1.336,0.132,0.895,-2.444,2.795



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,45.397,1.977


In [16]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 69.5225
Min. group size: 16
Log-Likelihood: -1164.1038
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,29.391,1.685,17.443,0.000,26.088,32.693
1,wh_numeric,0.130,0.741,0.175,0.861,-1.323,1.583
2,gap_numeric,3.085,0.741,4.161,0.000,1.632,4.538
3,wh_numeric:gap_numeric,0.005,0.741,0.007,0.995,-1.448,1.458
4,gap_distance_length,-0.011,0.016,-0.658,0.511,-0.043,0.021
5,wh_numeric:gap_distance_length,-0.001,0.016,-0.048,0.961,-0.033,0.031
6,gap_numeric:gap_distance_length,0.004,0.016,0.255,0.799,-0.028,0.036
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.016,0.005,0.996,-0.032,0.032



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,45.690,2.009


Embedded Clause

In [17]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [18]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [19]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2069
Min. group size: 16
Log-Likelihood: -260.4999
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.535,0.155,54.958,0.000,8.231,8.839
1,gap_distance[T.short],0.842,0.072,11.709,0.000,0.701,0.983
2,gap_distance[T.medium],0.986,0.072,13.714,0.000,0.845,1.127
3,gap_distance[T.long],1.869,0.072,25.988,0.000,1.728,2.010
4,wh_numeric,0.070,0.051,1.374,0.169,-0.030,0.170
5,wh_numeric:gap_distance[T.short],-0.045,0.072,-0.621,0.535,-0.186,0.096
6,wh_numeric:gap_distance[T.medium],-0.025,0.072,-0.346,0.730,-0.166,0.116
7,wh_numeric:gap_distance[T.long],-0.030,0.072,-0.416,0.678,-0.171,0.111
8,gap_numeric,-0.390,0.051,-7.660,0.000,-0.489,-0.290
9,gap_numeric:gap_distance[T.short],0.141,0.072,1.961,0.050,0.000,0.282



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.431,0.327


In [20]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.1973
Min. group size: 16
Log-Likelihood: -259.4066
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.629,0.147,58.564,0.000,8.340,8.918
1,wh_numeric,0.054,0.039,1.360,0.174,-0.024,0.131
2,gap_numeric,-0.362,0.039,-9.157,0.000,-0.439,-0.284
3,wh_numeric:gap_numeric,-0.005,0.039,-0.138,0.891,-0.083,0.072
4,gap_distance_length,0.023,0.001,26.830,0.000,0.022,0.025
5,wh_numeric:gap_distance_length,-0.000,0.001,-0.284,0.777,-0.002,0.001
6,gap_numeric:gap_distance_length,0.003,0.001,3.974,0.000,0.002,0.005
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.001,0.044,0.965,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.403,0.313


Embedded Clause - PP/Goal Gap Position

In [21]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [22]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2069
Min. group size: 16
Log-Likelihood: -257.7628
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.853,0.136,65.134,0.000,8.587,9.120
1,gap_distance[T.short],0.775,0.072,10.772,0.000,0.634,0.916
2,gap_distance[T.medium],0.871,0.072,12.106,0.000,0.730,1.012
3,gap_distance[T.long],1.712,0.072,23.805,0.000,1.571,1.853
4,wh_numeric,0.110,0.051,2.159,0.031,0.010,0.210
5,wh_numeric:gap_distance[T.short],-0.039,0.072,-0.549,0.583,-0.180,0.102
6,wh_numeric:gap_distance[T.medium],-0.033,0.072,-0.461,0.644,-0.174,0.108
7,wh_numeric:gap_distance[T.long],-0.043,0.072,-0.591,0.554,-0.184,0.098
8,gap_numeric,-0.099,0.051,-1.947,0.052,-0.199,0.001
9,gap_numeric:gap_distance[T.short],0.065,0.072,0.907,0.365,-0.076,0.206



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.318,0.244


In [23]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.1972
Min. group size: 16
Log-Likelihood: -256.5388
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.932,0.128,69.723,0.000,8.681,9.183
1,wh_numeric,0.097,0.039,2.464,0.014,0.020,0.175
2,gap_numeric,-0.087,0.039,-2.203,0.028,-0.164,-0.010
3,wh_numeric:gap_numeric,0.006,0.039,0.150,0.881,-0.071,0.083
4,gap_distance_length,0.021,0.001,24.588,0.000,0.020,0.023
5,wh_numeric:gap_distance_length,-0.000,0.001,-0.529,0.597,-0.002,0.001
6,gap_numeric:gap_distance_length,0.001,0.001,1.696,0.090,-0.000,0.003
7,wh_numeric:gap_numeric:gap_distance_length,-0.000,0.001,-0.089,0.929,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.297,0.233


#### Double Gaps

In [24]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [25]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)

Post Gap Region - Modifier

In [26]:
#sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['modifier']), axis = 1)
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['modifier'], 'object'), axis = 1)

In [27]:
iv_list = ['subject_gap', 'object_gap', 'wh_numeric']
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'double gap modifier')


=== DOUBLE GAP MODIFIER ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 19.9844
Min. group size: 8
Log-Likelihood: -475.2594
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,16.368,1.067,15.337,0.000,14.277,18.460
1,subject_gap,1.393,1.000,1.393,0.164,-0.566,3.352
2,object_gap,11.276,1.000,11.280,0.000,9.316,13.235
3,subject_gap:object_gap,0.730,1.414,0.516,0.606,-2.041,3.501
4,wh_numeric,0.085,0.707,0.120,0.904,-1.301,1.470
5,subject_gap:wh_numeric,-0.024,1.000,-0.024,0.981,-1.984,1.935
6,object_gap:wh_numeric,0.016,1.000,0.016,0.987,-1.943,1.975
7,subject_gap:object_gap:wh_numeric,0.070,1.414,0.050,0.960,-2.701,2.841



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,12.789,1.186


Subject Gap Position

In [28]:
subjectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'b', 'd', 'e', 'f', 'h'])]

In [29]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(subjectgap_df, iv_list, 'region_surprisal', 'double gap modifier: subject gap position')


=== DOUBLE GAP MODIFIER: SUBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 48.2200
Min. group size: 6
Log-Likelihood: -401.7362
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.066,0.771,26.019,0.000,18.555,21.578
1,wh_numeric,0.094,0.672,0.140,0.889,-1.224,1.412
2,gap_numeric,3.698,0.672,5.500,0.000,2.380,5.016
3,wh_numeric:gap_numeric,0.009,0.672,0.014,0.989,-1.308,1.327



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.854,0.556


Object Gap Position

In [30]:
objectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'c', 'd', 'e', 'g', 'h'])]

In [31]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'double gap modifier: object gap position')


=== DOUBLE GAP MODIFIER: OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 19.2219
Min. group size: 6
Log-Likelihood: -364.6137
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.537,1.103,20.427,0.000,20.374,24.699
1,wh_numeric,0.104,0.425,0.245,0.806,-0.728,0.936
2,gap_numeric,6.169,0.425,14.531,0.000,5.336,7.001
3,wh_numeric:gap_numeric,0.019,0.425,0.046,0.964,-0.813,0.851



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,20.741,1.938


Embedded Clause

In [32]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [33]:
iv_list = ['subject_gap', 'object_gap', 'wh_numeric']
result = mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed')
result


=== DOUBLE GAP EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.1079
Min. group size: 8
Log-Likelihood: -90.9317
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.920,0.132,52.405,0.000,6.661,7.178
1,subject_gap,-0.187,0.073,-2.546,0.011,-0.331,-0.043
2,object_gap,-0.108,0.073,-1.465,0.143,-0.252,0.036
3,subject_gap:object_gap,-0.104,0.104,-0.997,0.319,-0.307,0.100
4,wh_numeric,0.076,0.052,1.468,0.142,-0.026,0.178
5,subject_gap:wh_numeric,0.035,0.073,0.473,0.636,-0.109,0.179
6,object_gap:wh_numeric,0.013,0.073,0.181,0.856,-0.131,0.157
7,subject_gap:object_gap:wh_numeric,0.012,0.104,0.112,0.911,-0.192,0.215



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.295,0.325


Subject Gap Position

In [34]:
subjectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'b', 'd', 'e', 'f', 'h'])]

In [35]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(subjectgap_df, iv_list, 'embed_surprisal', 'double gap embedded clause: subject gap position')


=== DOUBLE GAP EMBEDDED CLAUSE: SUBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1165
Min. group size: 6
Log-Likelihood: -75.1745
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.773,0.122,55.638,0.000,6.535,7.012
1,wh_numeric,0.100,0.033,3.022,0.003,0.035,0.165
2,gap_numeric,-0.146,0.033,-4.426,0.000,-0.211,-0.082
3,wh_numeric:gap_numeric,0.024,0.033,0.715,0.475,-0.041,0.088



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.275,0.306


Object Gap Position

In [36]:
objectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'c', 'd', 'e', 'g', 'h'])]

In [37]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(objectgap_df, iv_list, 'embed_surprisal', 'double gap embedded clause: object gap position')


=== DOUBLE GAP EMBEDDED CLAUSE: OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1238
Min. group size: 6
Log-Likelihood: -79.5099
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.793,0.131,51.926,0.000,6.537,7.050
1,wh_numeric,0.094,0.034,2.773,0.006,0.028,0.161
2,gap_numeric,-0.126,0.034,-3.710,0.000,-0.193,-0.060
3,wh_numeric:gap_numeric,0.018,0.034,0.536,0.592,-0.049,0.085



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.319,0.342


#### Wh-Islands

In [38]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [39]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)

Post-Gap Region - Continuation

In [40]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [41]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation')


=== WH-ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 36.2797
Min. group size: 12
Log-Likelihood: -778.9622
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,26.884,1.229,21.872,0.000,24.475,29.293
1,comp_type[T.that],0.010,0.952,0.010,0.992,-1.857,1.876
2,comp_type[T.whether],-0.072,0.952,-0.075,0.940,-1.938,1.795
3,wh_numeric,0.142,0.673,0.211,0.833,-1.178,1.462
4,wh_numeric:comp_type[T.that],-0.128,0.952,-0.135,0.893,-1.995,1.738
5,wh_numeric:comp_type[T.whether],-0.084,0.952,-0.088,0.929,-1.951,1.782
6,gap_numeric,5.745,0.673,8.531,0.000,4.425,7.065
7,gap_numeric:comp_type[T.that],0.043,0.952,0.045,0.964,-1.824,1.909
8,gap_numeric:comp_type[T.whether],-0.149,0.952,-0.157,0.875,-2.016,1.717
9,wh_numeric:gap_numeric,-0.106,0.673,-0.157,0.875,-1.426,1.214



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,21.148,1.360


Embedded Clause

In [42]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [43]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed')


=== WH-ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0292
Min. group size: 12
Log-Likelihood: -3.1243
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.321,0.212,44.007,0.000,8.906,9.736
1,comp_type[T.that],-0.169,0.027,-6.268,0.000,-0.222,-0.116
2,comp_type[T.whether],0.113,0.027,4.183,0.000,0.060,0.166
3,wh_numeric,-0.153,0.019,-8.031,0.000,-0.191,-0.116
4,wh_numeric:comp_type[T.that],-0.021,0.027,-0.781,0.435,-0.074,0.032
5,wh_numeric:comp_type[T.whether],-0.026,0.027,-0.970,0.332,-0.079,0.027
6,gap_numeric,0.059,0.019,3.085,0.002,0.022,0.096
7,gap_numeric:comp_type[T.that],-0.010,0.027,-0.362,0.718,-0.063,0.043
8,gap_numeric:comp_type[T.whether],0.008,0.027,0.308,0.758,-0.045,0.061
9,wh_numeric:gap_numeric,-0.011,0.019,-0.566,0.571,-0.048,0.027



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.890,1.769


#### Adjunct Islands 

In [44]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [45]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [46]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)

Post-Gap Region - Continuation

In [47]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [48]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation')


=== ADJUNCT ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 48.4482
Min. group size: 12
Log-Likelihood: -813.8055
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,38.256,1.539,24.856,0.000,35.239,41.272
1,adjunct_type[T.back],-0.120,1.101,-0.109,0.913,-2.277,2.037
2,adjunct_type[T.front],-0.196,1.101,-0.178,0.859,-2.353,1.961
3,wh_numeric,0.057,0.778,0.073,0.942,-1.469,1.582
4,wh_numeric:adjunct_type[T.back],0.099,1.101,0.090,0.928,-2.058,2.256
5,wh_numeric:adjunct_type[T.front],0.039,1.101,0.036,0.972,-2.118,2.196
6,gap_numeric,4.579,0.778,5.884,0.000,3.054,6.104
7,gap_numeric:adjunct_type[T.back],-0.025,1.101,-0.023,0.982,-2.182,2.132
8,gap_numeric:adjunct_type[T.front],-0.461,1.101,-0.419,0.675,-2.618,1.696
9,wh_numeric:gap_numeric,-0.008,0.778,-0.010,0.992,-1.533,1.517



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,35.264,1.913


Embedded Clause

In [49]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [50]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed')


=== ADJUNCT ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.1275
Min. group size: 12
Log-Likelihood: -157.0617
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.935,0.213,41.901,0.000,8.517,9.353
1,adjunct_type[T.back],0.671,0.056,11.877,0.000,0.560,0.781
2,adjunct_type[T.front],0.744,0.056,13.176,0.000,0.633,0.855
3,wh_numeric,0.105,0.040,2.623,0.009,0.026,0.183
4,wh_numeric:adjunct_type[T.back],-0.016,0.056,-0.275,0.784,-0.126,0.095
5,wh_numeric:adjunct_type[T.front],0.021,0.056,0.376,0.707,-0.089,0.132
6,gap_numeric,-0.285,0.040,-7.141,0.000,-0.363,-0.207
7,gap_numeric:adjunct_type[T.back],0.103,0.056,1.827,0.068,-0.008,0.214
8,gap_numeric:adjunct_type[T.front],0.096,0.056,1.696,0.090,-0.015,0.206
9,wh_numeric:gap_numeric,0.006,0.040,0.138,0.890,-0.073,0.084



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.878,0.843


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands: subject condition')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands: object condition')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause subject condition')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause object condition')